In [72]:
"""
REUSE: Much of this code comes from the boilerplate code for the first problem
set for the Data-Intensive International Design course at UC Berkeley.
"""

import argparse
import time
import glob
from io import BytesIO
import urllib.request
import os.path
import sys

import numpy as np
from scipy import misc, ndimage
from osgeo import gdal, ogr, osr
import pandas as pd


# Shapefiles downloaded from here:
# http://www.gadm.org/download
COUNTRY_SHP_FILES = {
    # South America countries
    "Haiti": "D:/Intensive Data/Final Project/Country Shape File/shp/Haiti.shp",
    "Honduras": "D:/Intensive Data/Final Project/Country Shape File/shp/Honduras.shp",
    "Guatemala": "D:/Intensive Data/Final Project/Country Shape File/shp/Guatemala.shp",
    # East Africa countries
    "Rwanda": "D:/Intensive Data/Final Project/Country Shape File/shp/Rwanda.shp",
    "Burundi": "D:/Intensive Data/Final Project/Country Shape File/shp/Burundi.shp",
    # Asian countries
    "Nepal": "D:/Intensive Data/Final Project/Country Shape File/shp/Nepal.shp",
    "Bangladesh": "D:/Intensive Data/Final Project/Country Shape File/shp/Bangladesh.shp",
    "Nigeria": "D:/Intensive Data/Final Project/Country Shape File/shp/Nigeria.shp",
}


def _get_shp_file(country_name):
    return COUNTRY_SHP_FILES[country_name]


# Helper function to read a shapefile
def get_shp_extent(shp_file):
    """
    `shp_file`: Path to shapefile
    return:
    * Boundary location of the shapefile (x_min, x_max, y_min, y_max)
    * Geometry representing the SHP (can be used for containment checks)
    """
    inDriver = ogr.GetDriverByName("ESRI Shapefile")
    inDataSource = inDriver.Open(shp_file, 0)
    inLayer = inDataSource.GetLayer()
    feature = inLayer.GetFeature(0)
    geometry = feature.GetGeometryRef()
    extent = inLayer.GetExtent()
    # It's important to clone this geometry.  Otherwise, I think it might get
    # garbage collected before we can do anything useful with it?
    return extent, geometry.Clone()



# Helper function to get the pixel index of the point
def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    """
    Function
    --------
    get_cell_idx
    Given a point location and all the pixel locations of the raster file,
    get the column and row index of the point in the raster
    Parameters
    ----------
    lon : float
        Longitude of the point
    lat : float
        Latitude of the point
    top_left_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the top-left point in each pixel
    top_left_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the top-left point in each pixel
    
    Returns
    -------
    lon_idx : int
        Column index
    lat_idx : int
        Row index
    """
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx


# Helper function to read a raster file (just used to get lats / longs for now)
def read_raster(raster_file):
    """
    Function
    --------
    read_raster
    Given a raster file, get the pixel size, pixel location, and pixel value
    Parameters
    ----------
    raster_file : string
        Path to the raster file
    Returns
    -------
    x_size : float
        Pixel size
    top_left_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the top-left point in each pixel
    top_left_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the top-left point in each pixel
    centroid_x_coords : numpy.ndarray  shape: (number of columns,)
        Longitude of the centroid in each pixel
    centroid_y_coords : numpy.ndarray  shape: (number of rows,)
        Latitude of the centroid in each pixel
    bands_data : numpy.ndarray  shape: (number of rows, number of columns, 1)
        Pixel value
    """
    raster_dataset = gdal.Open(raster_file, gdal.GA_ReadOnly)
    # get project coordination
    proj = raster_dataset.GetProjectionRef()
    bands_data = []
    # Loop through all raster bands
    for b in range(1, raster_dataset.RasterCount + 1):
        band = raster_dataset.GetRasterBand(b)
        bands_data.append(band.ReadAsArray())
        no_data_value = band.GetNoDataValue()
    bands_data = np.dstack(bands_data)
    rows, cols, n_bands = bands_data.shape

    # Get the metadata of the raster
    geo_transform = raster_dataset.GetGeoTransform()
    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = geo_transform
    
    # Get location of each pixel
    x_size = 1.0 / int(round(1 / float(x_size)))
    y_size = - x_size
    y_index = np.arange(bands_data.shape[0])
    x_index = np.arange(bands_data.shape[1])
    top_left_x_coords = upper_left_x + x_index * x_size
    top_left_y_coords = upper_left_y + y_index * y_size
    # Add half of the cell size to get the centroid of the cell
    centroid_x_coords = top_left_x_coords + (x_size / 2)
    centroid_y_coords = top_left_y_coords + (y_size / 2)

    return (x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data)


def download_images(shape_file, country_name):

    # Read in the shapefile for the country, getting a rectangle for it, and a geometry
    # we can use to check for point containment.
    (x_min_shp, x_max_shp, y_min_shp, y_max_shp), countryGeometry = get_shp_extent(shape_file)

    # XXX: Use raster file to get the baseline longitudes and latitudes,
    # And then look up cell indexes for which we'll fetch images for the country.
    x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data = read_raster(raster_file)
    left_idx, top_idx = get_cell_idx(x_min_shp, y_max_shp, top_left_x_coords, top_left_y_coords)
    right_idx, bottom_idx = get_cell_idx(x_max_shp, y_min_shp, top_left_x_coords, top_left_y_coords)
    
    full_lightness = []
    full_i = []
    full_j = []
    full_lon = []
    full_lat = []
    full_df = pd.DataFrame()
    
    for i in range(left_idx, right_idx + 1):
        for j in range(top_idx, bottom_idx + 1):

            lon = centroid_x_coords[i]
            lat = centroid_y_coords[j]
            
            point = ogr.Geometry(ogr.wkbPoint)
            point.AddPoint(lon, lat)
            if countryGeometry.Contains(point):
                lightness = bands_data[j, i, 0]
                full_lightness.append(lightness)
                full_i.append(i)
                full_j.append(j)
                full_lon.append(lon)
                full_lat.append(lat)
    
    full_df['lightness'] = full_lightness
    full_df['full_i'] = full_i
    full_df['full_j'] = full_j
    full_df['full_lat'] = full_lat
    full_df['full_lon'] = full_lon

    return  full_df

# download_images(shp_path, args.country)


In [14]:
# this illustrates how you can read the nightlight image
raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182013.v4c_web.stable_lights.avg_vis.tif'
x_size, top_left_x_coords, top_left_y_coords, centroid_x_coords, centroid_y_coords, bands_data = read_raster(raster_file)
# save the result in compressed format - see https://docs.scipy.org/doc/numpy/reference/generated/numpy.savez.html
np.savez('nightlight2.npz', top_left_x_coords=top_left_x_coords, top_left_y_coords=top_left_y_coords, bands_data=bands_data)

In [28]:
import pandas as pd

raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182013.v4c_web.stable_lights.avg_vis.tif'
shape_file = _get_shp_file('Nepal')
nightlight = download_images(shape_file, 'Nepal')
nightlight['class'] = 3
nightlight['class'][nightlight['lightness'] <= 2] = 0
nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
nightlight['class'][nightlight['lightness'] >= 35] = 2
nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/bangladesh_TL.csv')

KeyboardInterrupt: 

In [36]:
import os
from osgeo import ogr, gdal

shape_file = _get_shp_file('Nigeria')
fname = shape_file # change this for your example

if not os.path.isfile(fname):
    raise IOError('Could not find file ' + str(fname))
source = ogr.Open(fname, gdal.GA_Update)
if source is None:
    raise IOError('Could not open file ' + str(fname))
...

Ellipsis

In [73]:
raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182013.v4c_web.stable_lights.avg_vis.tif'
shape_file = 'D:/Intensive Data/Final Project/Nigeria Gas Flare removed/NI.shp'
nightlight = download_images(shape_file, 'Nigeria')
nightlight['class'] = 3
nightlight['class'][nightlight['lightness'] <= 2] = 0
nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
nightlight['class'][nightlight['lightness'] >= 35] = 2
nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/nigeria2013_TL.csv')

# raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F162018.v4b_web.stable_lights.avg_vis.tif'
# shape_file = _get_shp_file('Nigeria')
# nightlight = download_images(shape_file, 'Nigeria')
# nightlight['class'] = 3
# nightlight['class'][nightlight['lightness'] <= 2] = 0
# nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
# nightlight['class'][nightlight['lightness'] >= 35] = 2
# nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/nigeria2008_TL.csv')


# raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182011.v4c_web.stable_lights.avg_vis.tif'
# shape_file = _get_shp_file('Nepal')
# nightlight = download_images(shape_file, 'Nepal')
# nightlight['class'] = 3
# nightlight['class'][nightlight['lightness'] <= 2] = 0
# nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
# nightlight['class'][nightlight['lightness'] >= 35] = 2
# nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/nepal_TL.csv')

# raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182013.v4c_web.stable_lights.avg_vis.tif'
# shape_file = _get_shp_file('Haiti')
# nightlight = download_images(shape_file, 'Haiti')
# nightlight['class'] = 3
# nightlight['class'][nightlight['lightness'] <= 2] = 0
# nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
# nightlight['class'][nightlight['lightness'] >= 35] = 2
# nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/haiti_TL.csv')

# raster_file = 'D:/Intensive Data/Final Project/Nightlight Image/F182012.v4c_web.stable_lights.avg_vis.tif'
# shape_file = _get_shp_file('Honduras')
# nightlight = download_images(shape_file, 'Honduras')
# nightlight['class'] = 3
# nightlight['class'][nightlight['lightness'] <= 2] = 0
# nightlight['class'][(nightlight['lightness'] > 2)&(nightlight['lightness'] < 35)] = 1
# nightlight['class'][nightlight['lightness'] >= 35] = 2
# nightlight.to_csv('D:/Intensive Data/Final Project/Nightlights for TL/honduras_TL.csv')


C:\Users\ntran\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ntran\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ntran\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [91]:

df2 = pd.read_csv('D:/Intensive Data/Final Project/nigeria2013_water_educ_wealth.csv')
geo_clusters = pd.read_csv('D:/Intensive Data/Final Project/DHS Nightlight/DHS_nightlights_Nigeria.csv')
geo_clusters.DHSCLUST = geo_clusters.DHSCLUST.astype(int)
hh_geo_clusters = geo_clusters

# merge wealth index and coordinates
merged = df2.merge(hh_geo_clusters, left_on='HV001', right_on='DHSCLUST')
# merged.drop('cluster',axis=1, inplace=True)

cluster_centers = merged.groupby(['DHSCLUST'], as_index = False).mean()
cluster_centers.columns = cluster_centers.columns.str.replace('DHSCLUST','id')
cluster_centers.columns = cluster_centers.columns.str.replace('water_index','avg_water_index')


In [93]:
k = cluster_centers[['id','xcoord','ycoord','wealth_index','max_','mean_','median_','min_','std_']]
k.to_csv('D:/Intensive Data/Final Project/Nigeria2013_wealth.csv', index = False)